In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cifar10
import cifar10_input

In [2]:

filename_queue = tf.train.string_input_producer(
tf.train.match_filenames_once("./images/6.jpg"))

    # Read an entire image file which is required since they're JPEGs, if the images
    # are too large they could be split in advance to smaller files or use the Fixed
    # reader to split up the file.
image_reader = tf.WholeFileReader()

    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
_,image_file = image_reader.read(filename_queue)

label = tf.Variable(6)
    
    # Decode the image as a JPEG file, this will turn it into a Tensor which we can
    # then use in training.
image = tf.image.decode_jpeg(image_file)

    # Start a new session to show example output.


In [3]:
with tf.Session() as sess:
        # Required to get the filename matching to run.
    init = (tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run([init])
   

        # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    my_img = image.eval()
        # Get an image tensor and print its value.
    image_tensor = sess.run([image])
    label_tensor = sess.run([label])
        # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)
        

In [4]:
Image.fromarray(np.asarray(my_img)).show()

In [5]:
reshaped_image = image

height = 24
width = 24

  # Image processing for evaluation.
  # Crop the central [height, width] of the image.


In [6]:
resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,height, width)



In [7]:
  # Subtract off the mean and divide by the variance of the pixels.
float_image = tf.image.per_image_standardization(resized_image)



In [8]:
  # Set the shapes of tensors.
float_image.set_shape([height, width, 3])

In [9]:
tf.summary.image('image2', float_image)

<tf.Tensor 'image2:0' shape=() dtype=string>

In [10]:
min_fraction_of_examples_in_queue = 0.4
num_examples_per_epoch = 10000
batch_size = 128
label2 = tf.convert_to_tensor(label_tensor)
min_queue_examples = int(num_examples_per_epoch * min_fraction_of_examples_in_queue)
  # Generate a batch of images and labels by building up a queue of examples.

images, labels = cifar10_input._generate_image_and_label_batch(float_image, label2,
                                         min_queue_examples, batch_size,
                                         shuffle=False)

In [11]:
logits = cifar10.inference(images)